In [1]:
import pandas as pd
import numpy as np
from utils import *
data = pd.read_excel('/home/abdelrahman.sadallah/mbzuai/review_rewrite/chatgpt/outputs/deepseek-r1-qwen-32b_gold_human_annotations_results.xlsx', sheet_name=None)


In [2]:
data['verifiability_extraction']['predicted_score']

0      yes
1      yes
2      yes
3      yes
4      yes
      ... 
139    yes
140    yes
141    yes
142    yes
143    yes
Name: predicted_score, Length: 144, dtype: object

In [3]:
data['verifiability_verification']

,review_point,paper_id,venue,focused_review,actionability,actionability_label,actionability_label_type,batch,grounding_specificity,grounding_specificity_label,...,professional_tone,professional_tone_label,professional_tone_label_type,valid_point,valid_point_label,valid_point_label_type,id,prompt,generation,predicted_score
0,2) it only compares with some weak baselines i...,ARR_2022_123_review,ARR_2022,1) it uses different experiment settings (e.g....,"['3', '5', '1']",NaN,NaN,1,"['4', '5', '5']",5.0,...,"['1', '1', '1']",1,gold,"['0', '0']",0,silver,1,<｜begin▁of▁sentence｜><｜User｜>\nYou are an expe...,"Okay, so I need to evaluate this review point ...",5
1,2. The evaluation on the word analogy task see...,ACL_2017_318_review,ACL_2017,1. Presentation and clarity: important details...,"['1', '4', '1', '4', '2', '1', '2']",1.0,gold,2,"['4', '5', '5', '4', '1', '1', '5']",5.0,...,"['1', '1', '1', '1', '1', '1', '1']",1,gold,"['1', '0', '1', '0', '0', '0', '0']",0,gold,2,<｜begin▁of▁sentence｜><｜User｜>\nYou are an expe...,"Okay, so I need to evaluate the verifiability ...",3
2,"- Also, since the dataset is artificially crea...",ARR_2022_236_review,ARR_2022,"- My main criticism is that the ""mismatched"" i...","['2', '3', '3', '4', '5', '3', '2', '5']",3.0,gold,2,"['2', '4', '4', '4', '5', '5', '2', '3']",4.0,...,"['1', '1', '1', '1', '1', '1', '1', '1']",1,gold,"['0', '0', '0', '0', '0', '0', '0']",0,gold,3,<｜begin▁of▁sentence｜><｜User｜>\nYou are an expe...,"Okay, so I need to evaluate this review point ...",3
3,1) The paper does not dig into the theory prof...,ACL_2017_554_review,ACL_2017,1) The paper does not dig into the theory prof...,"['1', '2', '3', '2', '3', '1', '1', '2']",1.0,gold,2,"['2', '2', '4', '2', '2', '1', '1', '1']",2.0,...,"['1', '1', '1', '1', '1', '1', '1', '1']",1,gold,"['0', '0', '0', '0', '0', '0', '0', '0']",0,gold,4,<｜begin▁of▁sentence｜><｜User｜>\nYou are an expe...,"Okay, so I need to evaluate the verifiability ...",1
4,- 261&272: any reason you did not consider the...,ACL_2017_516_review,ACL_2017,Missing related work on anchor words Evaluatio...,"['1', '5', '4', '1', '5', '3', '1', '5']",1.0,gold,2,"['5', '5', '5', '5', '5', '5', '5', '5']",5.0,...,"['1', '1', '1', '1', '1', '1', '1', '1']",1,gold,"['0', '0', '0', '0', '0', '0', '0']",0,gold,5,<｜begin▁of▁sentence｜><｜User｜>\nYou are an expe...,"Okay, so I need to evaluate the verifiability ...",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,- I wonder what is the total computational com...,NIPS_2020_486,NIPS_2020,- I wonder what is the total computational com...,"['4', '3', '4']",4.0,silver,6,"['2', '4', '3']",NaN,...,"['1', '1', '1']",1,gold,"['0', '0', '0']",0,gold,140,<｜begin▁of▁sentence｜><｜User｜>\nYou are an expe...,"Okay, so I need to evaluate this review point ...",2
140,"- The authors write “In the base IOI circuit, ...",FbZSZEIkEU,ICLR_2025,- The experimental reports are lacking in many...,"['4', '1', '2']",NaN,NaN,6,"['5', '5', '5']",5.0,...,"['1', '1', '1']",1,gold,"['0', '0', '0']",0,gold,141,<｜begin▁of▁sentence｜><｜User｜>\nYou are an expe...,"Okay, so I need to evaluate the verifiability ...",5
141,"- L15: Again, too vague, certain RNNs work wel...",NIPS_2018_917,NIPS_2018,- Results on bAbI should be taken with a huge ...,"['4', '4', '1']",4.0,silver,6,"['5', '5', '5']",5.0,...,"['1', '1', '1']",1,gold,"['0', '0', '0']",0,gold,142,<｜begin▁of▁sentence｜><｜User｜>\nYou are an expe...,"Okay, so I need to evaluate the verifiability ...",3
142,104 it is said that every kernel can be descri...,NIPS_2021_895,NIPS_2021,The description of the method is somewhat uncl...,"['5', '4', '5']",5.0,silver,6,"['5', '5', '5']",5.0,...,"['1', '1', '1']",1,gold,"['0', '0', '0']",0,gold,143,<｜begin▁of▁sentence｜><｜User｜>\nYou are an expe...,"Okay, so I need to evaluate the verifiability ...",5


In [4]:
data['helpfulness']['predicted_score'].value_counts()

predicted_score
3    105
4     55
2     13
5     12
1      2
Name: count, dtype: int64

In [5]:
aspects = [ 'actionability', 'grounding_specificity', 'helpfulness', 'verifiability_verification']


with open(f'qwen_r1_outputs.txt', 'w') as f:
    for aspect in aspects:

        label_column =  'verifiability' if aspect == 'verifiability_verification' else aspect
        ## remove rows with nan values
        data[aspect] = data[aspect].dropna(subset=['predicted_score'])
        f.write('*' * 20 + f'{aspect}'+ '*' * 20 + '\n')
        f.write('number of samples: ' + str(len(data[aspect])) + '\n')

        predcts = data[aspect]['predicted_score'].values
        labels = data[aspect][f'{label_column}_label'].values

        stat_dict = get_stats(predcts, labels, label_column)
 

        ## remove the acc from the dict, and for spearman, only include the correlation

        for k,v in stat_dict.items():
            if 'accuracy' in k:
                continue
            if 'spearman' in k:
                v = v[0]
            ## round the values
            if isinstance(v, float):
                v = round(v, 3)
            f.write(f'{k}: {v}\n')
        f.write('-'*50+'\n')
